In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
os.getcwd()
print(os.getcwd())

C:\Users\omprakash.bankol\Desktop\MARS_Russia\Analytical Dataset Creation_v1\Analytical Dataset Creation\Code


In [2]:
Nielsen_data_Choco = pd.read_csv(r"../Data/Raw Data/(BARS)MERGED 2017-2020 NIELSEN BARS DATA LONG FORMAT.csv")
Nielsen_data_GUM = pd.read_csv(r"../Data/Raw Data/(GUMS )Neilsen_Complete_data_long_format.csv")
Nielsen_data_Choco = Nielsen_data_Choco[['SKU','Week','1:Units (in 1000 PACKS)']]
Nielsen_data_GUM = Nielsen_data_GUM[['SKU','Week','1:Units (in 1000 PACKS)']]

Nielsen_data=Nielsen_data_Choco.append(Nielsen_data_GUM)
Nielsen_data["Nielsen_Units"] =(Nielsen_data["1:Units (in 1000 PACKS)"])*1000
print(Nielsen_data.head())


                               SKU       Week  1:Units (in 1000 PACKS)  \
0  TORERO BROWNI MCH V ROLL 0.18KG  W 2017 09                      NaN   
1  TORERO BROWNI MCH V ROLL 0.18KG  W 2017 10                      NaN   
2  TORERO BROWNI MCH V ROLL 0.18KG  W 2017 11                      NaN   
3  TORERO BROWNI MCH V ROLL 0.18KG  W 2017 12                      NaN   
4  TORERO BROWNI MCH V ROLL 0.18KG  W 2017 13                      NaN   

   Nielsen_Units  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  


In [3]:
#Price monitoring data at SKU level
price_data = pd.read_excel(r"../Process/Price_Monitoring/TPR calculation on PRice from Price Monitoring file.xlsx")
col_req=['Retailer','PPG','Week','Nielsen_SKU','Price(Price Montioring File)']
price_data=price_data[col_req]
price_data.rename(columns={'Nielsen_SKU':'SKU','Price(Price Montioring File)':'Price_Monitoring'},inplace=True)

price_data['Year']=(price_data['Week'].str.slice(0,4))
price_data['tmp']=(price_data['Week'].str.slice(6,8))

del price_data['Week']
price_data['Week']='W '+(price_data['Year'])+' '+price_data['tmp']

del price_data['Year']
del price_data['tmp']

print(price_data.head())

  Retailer        PPG                                                SKU  \
0       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
1       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
2       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
3       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
4       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   

   Price_Monitoring       Week  
0               NaN  W 2017 01  
1               NaN  W 2017 02  
2               NaN  W 2017 03  
3               NaN  W 2017 04  
4               NaN  W 2017 05  


In [4]:
price_data1 = pd.merge(price_data,Nielsen_data, on=["SKU","Week"], how='left')
print(price_data1.shape)
print(price_data.shape)
#print(price_data['Nielsen_Units'])
price_data1.head()
#price_data1['Nielsen_Units']
print(price_data1['Nielsen_Units'].isna().sum())

(10498, 7)
(10498, 5)
3472


In [5]:
price_data1.dtypes

Retailer                    object
PPG                         object
SKU                         object
Price_Monitoring           float64
Week                        object
1:Units (in 1000 PACKS)    float64
Nielsen_Units              float64
dtype: object

In [6]:
week_mapping = pd.read_excel(r"../Data/MappingFile//Week Mapping.xlsx", sheet_name="Mapping")
week_mapping.rename(columns={'Week Start date':'Date','Week':'Week_no'},inplace=True)


price_data1['Year'] = pd.to_numeric(price_data1['Week'].str.slice(2,6))
price_data1['Week_no'] = pd.to_numeric(price_data1['Week'].str.slice(7,9))
price_data1['Week_no'] = "W"+pd.to_numeric(price_data1['Week_no']).astype(str)

In [7]:
print(week_mapping.head())

        Date  Year Week_no Month_Name  Month
0 2017-01-01  2017      W1    January      1
1 2017-01-08  2017      W2    January      1
2 2017-01-15  2017      W3    January      1
3 2017-01-22  2017      W4    January      1
4 2017-01-29  2017      W5   February      2


In [8]:
price_data2 = pd.merge(price_data1,week_mapping, on=["Year", "Week_no"], how='left')
print(price_data2.shape)
print(price_data2.head())

(10498, 12)
  Retailer        PPG                                                SKU  \
0       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
1       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
2       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
3       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
4       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   

   Price_Monitoring       Week  1:Units (in 1000 PACKS)  Nielsen_Units  Year  \
0               NaN  W 2017 01                      NaN            NaN  2017   
1               NaN  W 2017 02                      NaN            NaN  2017   
2               NaN  W 2017 03                      NaN            NaN  2017   
3               NaN  W 2017 04                      NaN            NaN  2017   
4               NaN  W 2017 05                      NaN            NaN  2017   

  Week_no       Date Month_Name  Month  
0      W1

In [9]:
#Period filter
#price_data2.loc[(price_data2['Month_Name']=='May') & (price_data2['Year']==2020),]
price_data2 = price_data2.loc[(price_data2['Date']>='2018-06-17') & (price_data2['Date']<='2020-03-22') , ]
price_data2.reset_index(drop=True, inplace=True)
price_data2.head(2)

,Retailer,PPG,SKU,Price_Monitoring,Week,1:Units (in 1000 PACKS),Nielsen_Units,Year,Week_no,Date,Month_Name,Month
0,X5,ORBIT OTC,ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...,28.280421,W 2018 25,333.9,333900.0,2018,W25,2018-06-17,June,6
1,X5,ORBIT OTC,ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...,27.976216,W 2018 26,363.8,363800.0,2018,W26,2018-06-24,June,6


In [10]:
# drop na price rows
price_data3=price_data2.dropna(subset=['Price_Monitoring'])
price_data3.reset_index(inplace=True,drop=True)
print(price_data3.shape)
print(price_data2.shape)
print(price_data3.head())



(3151, 12)
(4650, 12)
  Retailer        PPG                                                SKU  \
0       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
1       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
2       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
3       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   
4       X5  ORBIT OTC  ORBIT(MARS) KLUB.-BANAN BANAN&STRAWB F 10P 0.0...   

   Price_Monitoring       Week  1:Units (in 1000 PACKS)  Nielsen_Units  Year  \
0         28.280421  W 2018 25                    333.9       333900.0  2018   
1         27.976216  W 2018 26                    363.8       363800.0  2018   
2         27.506471  W 2018 27                    228.1       228100.0  2018   
3         27.566250  W 2018 28                    233.6       233600.0  2018   
4         28.788679  W 2018 29                    245.2       245200.0  2018   

  Week_no       Date Month_Name  Month  

In [11]:
#Fill NA 
price_data3['Nielsen_Units'] = price_data3['Nielsen_Units'].fillna(0)
#price_data3['Nielsen_Units'].isna().sum()
price_data3['PricexUnit']=(price_data3['Price_Monitoring'])*(price_data3['Nielsen_Units'])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
price_data4 = price_data3.groupby(['Retailer','PPG','Date']).aggregate({"Nielsen_Units":np.sum,'PricexUnit':np.sum,"Price_Monitoring":np.mean})
price_data4.reset_index(inplace=True,drop=False)
#price_data3.columns

In [13]:
#weighted price and average price calculation
price_data4['Weighted_Price_Monitoring']=price_data4['PricexUnit']/price_data4['Nielsen_Units']
price_data4.rename(columns={'Price_Monitoring':'Avg_Price_Monitoring'},inplace=True)
price_data4.head()

price_data4['Date']=pd.to_datetime(price_data4['Date']).dt.date


In [14]:
#############Median Base Price using Weighted Price Monitoring ##########
price_data4.sort_values(['Retailer','PPG','Date'], axis = 0, ascending = True,inplace = True)
price_data4.reset_index(drop=True,inplace=True)


for i in range(1,8):
    price_data4['Weighted_Price_Monitoring'+'_lag'+str(i)]=price_data4.groupby(['Retailer','PPG'])['Weighted_Price_Monitoring'].shift(periods=i)

#max of previous 7 weeks
Cols_Nielsen = [i for i in list(price_data4.columns) if "Weighted_Price_Monitoring_lag" in i] 
price_data4['Weighted_Price_Monitoring_prev']=  price_data4[Cols_Nielsen].max(axis=1)  


#sudden jump in price will map to NA
for i in range(1,8):
    var='Weighted_Price_Monitoring'+'_lag'+str(i)
    price_data4.loc[np.abs(price_data4[var]-price_data4['Weighted_Price_Monitoring_prev'])/price_data4['Weighted_Price_Monitoring_prev'] > 0.05,var]=np.nan


#median base price
#Weighted Median Price==WPM
price_data4['WPM_median_base_price']=price_data4[Cols_Nielsen].median(axis=1) 
#price_data4['WPM_median_base_price']=price_data4['WPM_median_base_price']

var='WPM_median_base_price'
idx = np.where(np.abs(price_data4[var]-price_data4['Weighted_Price_Monitoring'])/price_data4[var] <= 0.05, 1, 0)
price_data4.loc[idx==1,var]=price_data4.loc[idx==1,'Weighted_Price_Monitoring']
price_data4.loc[idx==0,var]=price_data4.loc[idx==0,var]


#final base price (more smoother than median base)
price_data4.sort_values(['Retailer','PPG','Date'], axis = 0, ascending = True,inplace = True)
price_data4.index=price_data4['Date']
final_base_price=price_data4.groupby(['Retailer','PPG'])[var].rolling(window=26,min_periods=1).max()
final_base_price=pd.DataFrame(final_base_price)
(final_base_price).reset_index(inplace=True)
final_base_price.rename(columns={'WPM_median_base_price':'WPM_final_base_price'},inplace=True)
price_data4.reset_index(inplace=True,drop=True)
price_data4=pd.merge(price_data4,final_base_price,on=['Retailer','PPG','Date'],how='left')

#na in final base price, replace by asp
idx = np.where(price_data4['WPM_final_base_price'].isna(), 1, 0)
price_data4.loc[idx==1,'WPM_final_base_price']=price_data4.loc[idx==1,'Weighted_Price_Monitoring']


#
var='WPM_final_base_price'
idx = np.where(np.abs(price_data4[var]-price_data4['WPM_median_base_price'])/price_data4[var] >= 0.05, 1, 0)
price_data4.loc[idx==1,'WPM_median_base_price']=price_data4.loc[idx==1,var]*0.95

Cols_Nielsen = [i for i in list(price_data4.columns) if "Weighted_Price_Monitoring_lag" in i] 
Cols_Nielsen=Cols_Nielsen+['Weighted_Price_Monitoring_prev']
price_data4.drop(Cols_Nielsen,axis=1,inplace=True)




In [15]:
price_data4.head()

,Retailer,PPG,Date,Nielsen_Units,PricexUnit,Avg_Price_Monitoring,Weighted_Price_Monitoring,WPM_median_base_price,WPM_final_base_price
0,Magnit,BIG BARS,2018-06-17,590600.0,2.980729e+07,50.296021,50.469512,NaN,50.469512
1,Magnit,BIG BARS,2018-06-24,517600.0,2.618504e+07,50.378424,50.589326,50.589326,50.589326
2,Magnit,BIG BARS,2018-07-01,519100.0,2.572688e+07,49.394861,49.560551,49.560551,50.589326
3,Magnit,BIG BARS,2018-07-08,515900.0,2.559875e+07,49.347939,49.619601,49.619601,50.589326
4,Magnit,BIG BARS,2018-07-15,506800.0,2.456393e+07,49.018357,48.468686,48.468686,50.589326


In [16]:
##### ##########Median base price using AVG_Price_Monitoring############
for i in range(1,8):
    price_data4['Avg_Price_Monitoring'+'_lag'+str(i)]=price_data4.groupby(['Retailer','PPG'])['Avg_Price_Monitoring'].shift(periods=i)



#max of previous 7 weeks
Cols_Nielsen = [i for i in list(price_data4.columns) if "Avg_Price_Monitoring_lag" in i] 
price_data4['Avg_Price_Monitoring_prev']=  price_data4[Cols_Nielsen].max(axis=1)  


#sudden jump in price will map to NA
for i in range(1,8):
    var='Avg_Price_Monitoring'+'_lag'+str(i)
    price_data4.loc[np.abs(price_data4[var]-price_data4['Avg_Price_Monitoring_prev'])/price_data4['Avg_Price_Monitoring_prev'] > 0.05,var]=np.nan


#median base price
#Average Median Price==APM
price_data4['APM_median_base_price']=price_data4[Cols_Nielsen].median(axis=1) 
#price_data4['APM_median_base_price']=price_data4['APM_median_base_price']


var='APM_median_base_price'
idx = np.where(np.abs(price_data4[var]-price_data4['Avg_Price_Monitoring'])/price_data4[var] <= 0.05, 1, 0)
price_data4.loc[idx==1,var]=price_data4.loc[idx==1,'Avg_Price_Monitoring']
price_data4.loc[idx==0,var]=price_data4.loc[idx==0,var]

#final base price (more smoother than median base)
price_data4.sort_values(['Retailer','PPG','Date'], axis = 0, ascending = True,inplace = True)
price_data4.index=price_data4['Date']
final_base_price=price_data4.groupby(['Retailer','PPG'])[var].rolling(window=26,min_periods=1).max()
final_base_price=pd.DataFrame(final_base_price)
(final_base_price).reset_index(inplace=True)
final_base_price.rename(columns={'APM_median_base_price':'APM_final_base_price'},inplace=True)
price_data4.reset_index(inplace=True,drop=True)
price_data4=pd.merge(price_data4,final_base_price,on=['Retailer','PPG','Date'],how='left')



#na in final base price, replace by asp
idx = np.where(price_data4['APM_final_base_price'].isna(), 1, 0)
price_data4.loc[idx==1,'APM_final_base_price']=price_data4.loc[idx==1,'Avg_Price_Monitoring']


#
var='APM_final_base_price'
idx = np.where(np.abs(price_data4[var]-price_data4['APM_median_base_price'])/price_data4[var] >= 0.05, 1, 0)
price_data4.loc[idx==1,'APM_median_base_price']=price_data4.loc[idx==1,var]*0.95

Cols_Nielsen = [i for i in list(price_data4.columns) if "Avg_Price_Monitoring_lag" in i] 
Cols_Nielsen=Cols_Nielsen+['Avg_Price_Monitoring_prev']
price_data4.drop(Cols_Nielsen,axis=1,inplace=True)




In [17]:
price_data4.head()

,Retailer,PPG,Date,Nielsen_Units,PricexUnit,Avg_Price_Monitoring,Weighted_Price_Monitoring,WPM_median_base_price,WPM_final_base_price,APM_median_base_price,APM_final_base_price
0,Magnit,BIG BARS,2018-06-17,590600.0,2.980729e+07,50.296021,50.469512,NaN,50.469512,NaN,50.296021
1,Magnit,BIG BARS,2018-06-24,517600.0,2.618504e+07,50.378424,50.589326,50.589326,50.589326,50.378424,50.378424
2,Magnit,BIG BARS,2018-07-01,519100.0,2.572688e+07,49.394861,49.560551,49.560551,50.589326,49.394861,50.378424
3,Magnit,BIG BARS,2018-07-08,515900.0,2.559875e+07,49.347939,49.619601,49.619601,50.589326,49.347939,50.378424
4,Magnit,BIG BARS,2018-07-15,506800.0,2.456393e+07,49.018357,48.468686,48.468686,50.589326,49.018357,50.378424


In [18]:

price_data4.to_excel(r"../Process/Price_Monitoring/Price_Monitoring_PPG_Data.xlsx",index=False)